In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rulesciation_rules

In [3]:
bb_df = pd.read_csv("../data/BB_2014.csv")

In [4]:
bb_df.head()

,SKU,Order,Delivery Date,Description
0,6883056,7998299,2014-04-23,Tide_Naturals Detergent Powder Lemon Chandan
1,6883056,6609767,2014-08-22,Tide_Naturals Detergent Powder Lemon Chandan
2,6883056,7963892,2014-05-05,Tide_Naturals Detergent Powder Lemon Chandan
3,6883056,7883032,2014-06-15,Tide_Naturals Detergent Powder Lemon Chandan
4,6883056,6474677,2014-09-25,Tide_Naturals Detergent Powder Lemon Chandan


In [6]:
len(bb_df.Order.unique())

32531

In [7]:
len(bb_df.SKU.unique())

6956

In [8]:
sku_desc_dict = dict(zip(bb_df.SKU, bb_df.Description))

In [9]:
bb_df.drop('Description', axis=1, inplace=True)

In [10]:
bb_df['Quantity'] =1

In [14]:
bb_pivot_df = bb_df.pivot(index='Order' ,
                          columns = 'SKU' ,
                          values = 'Quantity').reset_index(drop=True)

In [15]:
bb_pivot_df.head()

SKU,6883056,15668519,15668520,15668521,15668541,15668542,15670123,15670144,15670145,15670146,...,93302183,93302184,93302186,93303345,93303349,93316685,93316921,93316922,93320243,108525199
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
bb_pivot_df.fillna(0,inplace=True)

In [24]:
bb_pivot_df.shape

(32531, 6956)

In [25]:
frequent_itemsets = apriori(bb_pivot_df ,
                           min_support = 0.003,
                           use_colnames = True)

In [26]:
rules = association_rules(frequent_itemsets,
                         metric = "lift",
                         min_threshold=1)

In [27]:
rules.head()

,antecedants,consequents,support,confidence,lift
0,(15670251),(15670157),0.031877,0.111861,14.383220
1,(15670157),(15670251),0.007777,0.458498,14.383220
2,(34994312),(15670251),0.013925,0.256071,8.033013
3,(15670251),(34994312),0.031877,0.111861,8.033013
4,(34994313),(15670251),0.013741,0.219239,6.877605


In [41]:
def get_sku_description( skus ):

    all_skus = []

    for sku in skus:

         all_skus.append(sku_desc_dict[sku])

    return "".join(all_skus)        

In [42]:
rules['antecedants_desc'] = rules.antecedants.map(lambda rec: get_sku_description(rec))

In [43]:
rules['consequents_desc'] = rules.consequents.map(lambda rec: get_sku_description(rec))

In [44]:
rules.sort_values('confidence',ascending=False)

,antecedants,consequents,support,confidence,lift,antecedants_desc,consequents_desc
12,(34994312),(34994319),0.013925,0.514349,29.252238,TVS Organics_Potato,TVS Organics_Onion Regular
22,(93141092),(93141023),0.008761,0.470175,44.334137,Danone_Yoghurt Mango Flavour,Danone_Yoghurt Strawberry Flavour
1,(15670157),(15670251),0.007777,0.458498,14.383220,Gopalan Organic_Bhindi,Gopalan Organic_Spinach
21,(93141022),(93141092),0.008361,0.415441,47.420059,Danone_Yoghurt Vanilla Flavour,Danone_Yoghurt Mango Flavour
13,(34994319),(34994312),0.017583,0.407343,29.252238,TVS Organics_Onion Regular,TVS Organics_Potato
20,(93141092),(93141022),0.008761,0.396491,47.420059,Danone_Yoghurt Mango Flavour,Danone_Yoghurt Vanilla Flavour
23,(93141023),(93141092),0.010605,0.388406,44.334137,Danone_Yoghurt Strawberry Flavour,Danone_Yoghurt Mango Flavour
18,(93141022),(93141023),0.008361,0.371324,35.013118,Danone_Yoghurt Vanilla Flavour,Danone_Yoghurt Strawberry Flavour
9,(34936518),(34934493),0.023393,0.348226,10.270300,Fab 360_Fabric Washing Liquid,Bingo_Yumitos Original Style Salt Sprinkled
16,(34995028),(34995070),0.009775,0.327044,30.484439,Ambrosia_Capsicum Green,Ambrosia_Ladies Finger
